In [1]:
import pandas as pd
import numpy as np
import pylab as plt
from graph_tool.all import *
import unicodedata
%matplotlib inline

/opt/anaconda/anaconda3/lib/python3.5/site-packages/graph_tool/draw/cairo_draw.py:1480: RuntimeWarning: Error importing Gtk module: No module named 'gi'; GTK+ drawing will not work.
  warnings.warn(msg, RuntimeWarning)


In [ ]:
# #Get info from orbis
# info = [0, 17, 33, 43, 48, 57, 68, 69, 76, 78, 82, 91, 92, 93, 94]
# with open("Orbis_Export_1.csv") as f,open("nike_ownership.csv","w+") as fout:
#     flag= 0
#     for i,line in enumerate(f):
#         if "CONVERSE (ASIA PACIFIC) LTD" in line:
#             flag = 1
#         else:
#             pass
            
#         if (flag == 1) and (i < 20166):
#             print(unicodedata.normalize("NFKD", "\t".join(np.array(inf)[info])).split("\t"))            
            
#         if (flag == 1):
#             inf = line.split("\t")
#             #cols = [i for i,_ in enumerate(line.split("\t")) if len(_) > 0]
#             new_str = unicodedata.normalize("NFKD", "\t".join(np.array(inf)[info]))
#             fout.write(new_str)


df = pd.read_csv("./nike/nike_ownership.csv",sep="\t",header=None,na_values=["n.a.","-"],
             converters={9: lambda x: str(x)},names=["Index","Subsidiary","Country","Direct","Total","Revenue","Employees","City","TypeEntity","NACE1","NACE_description","Assets","BvD","PL","Level_ownership"],skip_footer=3,thousands=",")
df["Direct"] = df["Direct"].replace(d_own)
df["Total"] = df["Total"].replace(d_own)
df = df.dropna(subset=["Index"])
df

In [39]:
def process_company(filename):
    d_own = {'>50.00': 50.01, 'MO': 50.01, 'WO':99}
    df = pd.read_excel(filename,sheetname=1,na_values=["n.a.","-","Credit needed"],thousands=",")
    company = df["Company name"].values[0]
    df["Direct"] = df["Direct (%)"].replace(d_own)
    df["Total"] = df["Total (%)"].replace(d_own)
    df["Level"] = df["Level"].str.strip()

    d_index2bvd = dict(zip(df["Level"],df["Subsidiary BvD ID"]))
    d_index2bvd["0"] = company

    df["shareholder"] = df["Level"].apply(lambda x: x[:1+x[:-1].rfind(".")] if x[:-2].rfind(".") != -1 else "0")
    df["shareholder"] = df["shareholder"].apply(lambda x: d_index2bvd[x])
    df["Direct"] = df["Direct"].astype(float)
    df["Total"] = df["Total"].astype(float)

    edges = df[["Subsidiary BvD ID","shareholder","Direct","Total"]]
    edges["Type"] = "Directed"
    edges.columns = ["Target","Source","Weight","Total","Type"]


    edges["max"] = edges[["Weight","Total"]].max(1)
    edges = edges.replace(np.NaN,0)
    t = []
    for target,data in edges.groupby("Target"):
        if data["Weight"].sum() == 100:
            pass
        elif data["Weight"].sum() > 100:
            data["Weight"] = 100*data["Weight"]/np.sum(data["Weight"])
        else:
            data["Weight"] = data["Weight"] + data["max"]
            data["Weight"] = 100*data["Weight"]/np.sum(data["Weight"])
        t.append(data)
    edges = pd.concat(t)
    edges.sort_values(by="Weight",ascending=False)    

    nodes = df[["Subsidiary BvD ID","Subsidiary name","Country ISO code","Op.\nRevenue{0}*","No of\nemployees","Total\nAssets","P/L before tax","P/L for period"]]
    nodes.columns = ["Id","Label","Country","turnover","employees","assets","profit_before_tax","profit_after_tax"]
    nodes["profit_before_tax"] = nodes["profit_before_tax"].astype(float)
    nodes["profit_after_tax"] = nodes["profit_after_tax"].astype(float)

    edges.to_csv("./PietNikki/{}_edges.csv".format(company),sep="\t",index=False)
    nodes.drop_duplicates().to_csv("./PietNikki/{}_nodes.csv".format(company),sep="\t",index=False)

In [41]:
for i in range(1,12):
    if i == 5: continue
    print(i)
    process_company("./PietNikki/Orbis_Export_{}.xlsx".format(i))
    

6


/opt/anaconda/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/anaconda/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/anaconda/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

7
8
9
10
11


In [ ]:
g = Graph()